In [2]:
import random
from random import randint
import numpy as np 
import pandas as pd 
from faker import Faker
from faker.providers import BaseProvider

In [3]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [4]:
class CustomProvider:
    @classmethod
    def bdr_id(cls):
        faker = Faker()
        return faker.pystr_format("#########{{random_int}}") 

    @classmethod
    def iban_number(cls):
        faker = Faker()
        fake_iban = faker.country_code() + str(random_with_N_digits(2)) + faker.pystr(max_chars = 4).upper() + str(random_with_N_digits(6)) + str(random_with_N_digits(8))
        return fake_iban.upper() 
    
    @classmethod
    def id_class(cls):
        faker = Faker()
        return faker.pystr_format("#####{{random_int}}") 
    
    @classmethod
    def account_currency(cls):
        faker = Faker()
        return faker.pystr(max_chars = 3).upper()

    @classmethod
    def ten_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 10).upper()
    
    @classmethod
    def two_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 2).upper()
    
    @classmethod
    def one_letter(cls):
        faker = Faker()
        return faker.pystr(max_chars = 1).upper()
    
    @classmethod
    def num_compte(cls):
        faker = Faker()
        return faker.country_code() + str(random_with_N_digits(25))

In [5]:
faker = Faker()
faker.add_provider(CustomProvider())

In [6]:
faker.country_code()

'JP'

In [10]:
#getattr(faker.unique, faker.country_code())

In [8]:
test = []
for i in range(10000):
    test.append(getattr(faker.unique, 'iban_number')())

In [9]:
len(list(set(test)))

10000

In [4]:
def anonymizer(data, list_anonymised_dict):
    """
    Args:
        data (pandas.DataFrame)
            dataframe to anonymise
        list_anonymised_dict (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
        mapping (dictionary)
            mapping containing key as original value & value as anonymised value
    """
    mapping = {}
    for anonymised_dict in list_anonymised_dict:
        category = anonymised_dict['category']
        for col in anonymised_dict['col']:
            faker_unique_list = []
            unique_value = list((data[col]).unique())
            faker = Faker()
            faker.add_provider(CustomProvider())
            for i in range(len(unique_value)): 
                faker_unique_list.append(getattr(faker.unique, category)())
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            mapping[col] = mapping_anonymised
            data[col] = data[col].map(mapping_anonymised)
    return data, mapping

In [5]:
def anonymizer_multi(data_dict, anonymised_list):
    """
    Args:
        data_dict (dictionary of pandas.DataFrame)
            dictionary of dataframe to anonymise
        anonymised_list (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
    """
    mapping_multi = {}
    for table_name in data_dict:
        mapping_multi[table_name] = {}
    for anonymised_dict in anonymised_list:
        faker = Faker()
        faker.add_provider(CustomProvider())        
        for col in anonymised_dict['col']:
            print('col', col)
            unique_value = []
            faker_unique_list = []
            for table_name in anonymised_dict['table_name_list']:
                print('table', table_name)
                unique_value += list((data_dict[table_name][col]).unique())

                for i in range(len(unique_value)): 
                    faker_unique_list.append(getattr(faker.unique, anonymised_dict['category'])())
            
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            for table_name in anonymised_dict['table_name_list']:
                mapping_multi[table_name][col] = mapping_anonymised
                data_dict[table_name][col] = data_dict[table_name][col].map(mapping_anonymised)
    return data_dict, mapping_multi 

In [6]:
def anonymizer_iban_prepro(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_2_letters'] = data[col].str[:2]
        
    return data 

In [7]:
def anonymizer_iban_post(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_new'] = data[col]
        data = data.drop(columns = [col])
        data[col] = data[f'{col}_2_letters'] + data[f'{col}_new'].str[2:]
        print('Dropping col')
        data = data.drop(columns = [f'{col}_2_letters', f'{col}_new'])
    return data 

In [23]:
data_dict = {'test': ['a', 'b', 'c']}

In [24]:
df = pd.DataFrame(data_dict)

In [25]:
df

,test
0,a
1,b
2,c


In [41]:
list_company_name_account_anonym = ['test']

In [42]:
anonymised_dict_account = [{'col': list_company_name_account_anonym, 'category': 'company'}]

In [43]:
anonymised_dict_account

[{'col': ['test'], 'category': 'company'}]

In [44]:
anonymised_dict_account[0]['col']

['test']

In [45]:
for anonymised_dict in anonymised_dict_account:
    print(anonymised_dict)
    for col in anonymised_dict['col']:
        print('col:', col)

{'col': ['test'], 'category': 'company'}
col: test


In [46]:
(df, anonymised_dict_account)

(  test
 0    a
 1    b
 2    c,
 [{'col': ['test'], 'category': 'company'}])

In [52]:
mapping = {}
for anonymised_dict in anonymised_dict_account:
    print('anonymised_dict:', anonymised_dict)
    category = anonymised_dict['category']
    for col in anonymised_dict['col']:
        print('col:', col)
        faker_unique_list = []
        unique_value = list((df[col]).unique())
        faker = Faker()
        faker.add_provider(CustomProvider())
        for i in range(len(unique_value)): 
            print('category', category)
            faker_unique_list.append(getattr(faker.unique, category)())
        mapping_anonymised = dict(zip(unique_value, faker_unique_list))
        mapping[col] = mapping_anonymised
        df[col] = df[col].map(mapping_anonymised)


anonymised_dict: {'col': ['test'], 'category': 'company'}
col: test
category company
category company
category company


In [53]:
for i in range(len(unique_value)): 
    print('category')
    faker_unique_list.append(getattr(faker.unique, 'company')())

category
category
category


In [95]:
column_name = 'num_compte'
category_name = 'num_compte'
type = 'string'

In [106]:
num_rows = 10000
schema_list = [
{'column': 'name', 'category': 'name', 'type': type, 'unique': False},
{'column': 'address', 'category': 'address', 'type': type, 'unique': False},
{'column': 'job', 'category': 'job', 'type': type, 'unique': False},
{'column': 'company', 'category': 'company', 'type': type, 'unique': False},
]

In [107]:
class FakeGenerator():
    def __init__(self, num_rows, schema_list):
        self.num_rows : int = num_rows
        self.schema_list : list = schema_list

    def generate_fake_data(self):
        data = pd.DataFrame()
        for schema in self.schema_list:
            faker_list = []
            faker = Faker()
            faker.add_provider(CustomProvider())
            for i in range(self.num_rows): 
                faker_list.append(getattr(faker, schema['category'])())
            data[schema['column']] = faker_list
        return data

In [108]:
FakeGenerator(num_rows, schema_list).generate_fake_data()

,name,address,job,company
0,Samuel Mitchell,Unit 4924 Box 5871\nDPO AA 08650,Chief Technology Officer,Floyd Group
1,Ashley Dawson,"5521 Rachel Lodge\nAnthonyland, WY 15988",Local government officer,"Huber, Brown and Coleman"
2,Shawn Small,"2943 Justin Squares\nRebeccafurt, KS 51686",Biomedical engineer,Lambert PLC
3,Dr. Raymond Sawyer,"386 Stephen Forge\nJacksonton, CO 68946",Higher education careers adviser,"Reed, Snow and Lawson"
4,Arthur Ellis,"44998 Dana Brooks\nMathewsview, SC 28353",Clinical molecular geneticist,Wallace Ltd
...,...,...,...,...
9995,Melinda Curry,"9056 Melinda Pass Suite 263\nWest Brenda, CT 5...",Volunteer coordinator,Lawson Ltd
9996,Kathryn Lane,"571 Hunt Spring\nVeronicaton, FL 30654",Accommodation manager,"Logan, Fisher and Ross"
9997,Andrea Herman,USNS Morris\nFPO AA 03873,Textile designer,"Decker, Madden and Hamilton"
9998,Mary Glass,"556 Kelley Roads Apt. 262\nPort Melissaland, S...","Teacher, adult education",Martinez-Ali


### List of Category Faker 
- Faire une liste des categories existantes dans Faker 

**address**
- address
- building_number
- city
- city_suffix
- country
- country_code
- current_country
- current_country_code()
- postcode
- street_address
- street_name
- street_suffix

**bank**
- aba
- bank_country
- bban
- iban
- swift
- swift8
- swift11

**company**
- bs
- catch_phrase
- company
- company_suffix

**currency:**
- cryptocurrency
- cryptocurrency_code
- cryptocurrency_name
- currency_code
- currency_name
- currency_symbol
- pricetag

**phone_number:**
- country_calling_code
- msisdn
- phone_number

**person:**
- first_name
- first_name_female
- first_name_male
- language_name
- last_name
- name
- suffix
- job

**geo:**
- coordinate
- latitude
- latlng
- longitude

**credit_card_number:**
- credit_card_expire
- credit_card_full
- credit_card_number
- credit_card_provider
- credit_card_security_code

### Unique Value 

In [62]:
getattr(faker.unique, 'name')()

'Miranda King'